In [11]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [12]:
data = pd.read_csv('simulated_nz_ocean_current.csv')
data.head()

,SurveyId,Region,SurveyDate,SurveyAreaLatitudeStart,SurveyAreaLongitudeStart,Rubbish_Count,count_same,date average,Current_Speed,Current_Direction,Water_Temperature
0,1,Northland,09/10/2023,-34.334773,173.398059,92,112,0.922314,0.549671,150.267961,15.026004
1,2,Gisborne,12/08/2021,-34.052894,166.825654,101,112,1.497980,0.486174,79.958812,17.907068
2,3,Wellington,08/27/2019,-46.273675,166.478684,80,112,1.549685,0.564769,43.151532,14.470686
3,4,Northland,03/04/2023,-37.418538,167.740078,87,112,1.986591,0.652303,121.541462,20.440338
4,5,Gisborne,03/19/2021,-39.903095,166.177736,112,112,2.133945,0.476585,339.447493,16.251335


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SurveyId                  100 non-null    int64  
 1   Region                    100 non-null    object 
 2   SurveyDate                100 non-null    object 
 3   SurveyAreaLatitudeStart   100 non-null    float64
 4   SurveyAreaLongitudeStart  100 non-null    float64
 5   Rubbish_Count             100 non-null    int64  
 6   count_same                100 non-null    int64  
 7   date average              100 non-null    float64
 8   Current_Speed             100 non-null    float64
 9   Current_Direction         100 non-null    float64
 10  Water_Temperature         100 non-null    float64
dtypes: float64(6), int64(3), object(2)
memory usage: 8.7+ KB


In [14]:
#encode the behaviour column
le = LabelEncoder()
data['Region'] = le.fit_transform(data['Region'])

In [15]:
# Drop the 'SurveyId' and 'SurveyDate' columns
data = data.drop(columns=['SurveyId', 'SurveyDate'])
X = data.drop('Rubbish_Count', axis=1)
y = data['Rubbish_Count']

In [17]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(xtrain)
X_test_scaled = scaler.transform(xtest)